In [79]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

# Scraping a property page
This is a discovery notebook. It contains script that scrape a property page of the website seloger.com.\
From a page we can extract several features of the flat:

- Title
- URL (used a primary key)
- nom du quartier
- arrondissement/ville
- loyer (all inclusive)
- taxes and fees
- caution
- evolution des prix (si disponible)
- nb de pieces
- nb de m2
- "L'avis du professionel"
- Description de l'appart (general, interieur et autres)
- Diagnostic de performance energetique (note et kWhEP/m².an)

In [7]:
!pip install selenium

     |████████████████████████████████| 904 kB 1.9 MB/s eta 0:00:01


In [12]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
from bs4 import BeautifulSoup
from time import sleep
import datetime

In [72]:
opts = Options()

In [73]:
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36")
browser = webdriver.Chrome('../../../chromedriver', chrome_options=opts)
browser.set_page_load_timeout(30)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [74]:
seloger='https://www.seloger.com/annonces/locations/appartement/paris-20eme-75/pere-lachaise-reunion/164290613.htm'   


In [75]:
browser.get('https://www.seloger.com')
browser.get(seloger)  
sleep(5)

In [76]:
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
url = browser.current_url

In [77]:
print("checking for bot detection on %s" %url)
soup = BeautifulSoup(html_source, 'html.parser')
try:
    print(soup.find('div', {'class':'sc-bxivhb kOwDzU'}).text)
#     return soup.find('div', {'class':'sc-bxivhb kOwDzU'}).text == "Il semble que vous êtes nombreux à vous connecter depuis ce réseau ..."
#     print "\nbot detection!"
except:
    print(False)

checking for bot detection on https://www.seloger.com/annonces/locations/appartement/paris-20eme-75/pere-lachaise-reunion/164290613.htm
False


#### Comment:
Adding headers enables to avoid being detected as a bot when querying the webpage. When querying this website without headers, the response contains a captcha

# Navigate the soup
Most of the key information is in the **main** section of the HTML

In [618]:
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,'referer':'https://www.google.com/'}
response = requests.get(url, headers=header)
soup = BeautifulSoup(response.text,"html.parser")

## Title

In [80]:
soup.title.string

'Location Appartement 2 pièces Paris 20ème - Appartement F2/T2/2 pièces 34m² 1290€/mois - SeLoger'

## URL

In [81]:
for val in soup.find_all('meta'):
    if val.get('name') == 'description':
        print(val.get('content'))
        print(val.link.get('href'))

Rue de Bagnolet, à 150 mètres du métro Alexandre Dumas.
Appartement de 34 m² au 4ème et dernier étage (sans ascenseur) sur cour, calme et paisible, dans copropriété bien entretenue, double vitrage, tout équipé, comprenant:
- séjour (canapé lit,


AttributeError: 'NoneType' object has no attribute 'get'

In [82]:
soup.find('meta', attrs={'property':'og:url'}).get('content')

'https://www.seloger.com/annonces/locations/appartement/paris-20eme-75/pere-lachaise-reunion/164290613.htm'

In [83]:
soup.prettify()

'<html class="macos desktop landscape" lang="fr">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n  <meta content="telephone=no" name="format-detection"/>\n  <title>\n   Location Appartement 2 pièces Paris 20ème - Appartement F2/T2/2 pièces 34m² 1290€/mois - SeLoger\n  </title>\n  <meta content="Rue de Bagnolet, à 150 mètres du métro Alexandre Dumas.\r\nAppartement de 34 m² au 4ème et dernier étage (sans ascenseur) sur cour, calme et paisible, dans copropriété bien entretenue, double vitrage, tout équipé, comprenant:\r\n- séjour (canapé lit," name="description"/>\n  <meta content="Rue de Bagnolet, à 150 mètres du métro Alexandre Dumas.\r\nAppartement de 34 m² au 4ème et dernier étage (sans ascenseur) sur cour, calme et paisible, dans copropriété bien entretenue, double vitrage, tout équipé, comprenant:\r\n- séjour (canapé lit," name="og:description"/>\n  <link href="https://www.seloger.com/annonces/locations/appartement/pa

# Split soup into blocks

In [84]:
main_tag = soup.find_all('div',class_=re.compile("^app__CWrapMain-aroj7e-3"))[0]

In [85]:
summary_block = main_tag.find_all('div', class_=re.compile("^Summarystyled__MainWrapper-tzuaot-1"))[0]

In [86]:
quartier_block = (
    main_tag.find('p', class_=re.compile("^Map__AddressLine-sc-6i077b-2"))
)

In [87]:
quartier_block

<p class="Map__AddressLine-sc-6i077b-2 eCDRsF">Le bien est situé<!-- --> <strong>à <!-- -->Paris<!-- --> (<!-- -->75020<!-- -->)</strong>, dans le quartier <!-- -->Père Lachaise-Réunion<!-- -->.</p>

In [88]:
price_block = (
    main_tag
    .find('section', attrs={'data-test':'price-block', 'id':"a-propos-de-ce-prix"})
)

In [94]:
main_tag.find_all('section', attrs={'data-test':'price-block'})

[<section class="Pricestyled__Container-uc7t2j-1 cpdMqv" data-test="price-block" id=""><div class="Pricestyled__LeftColumn-uc7t2j-2 hJjRAQ"><div class="Pricestyled__Price-uc7t2j-0 cURZmh">Le loyer mensuel est de<div>1 290 €</div></div><div class="Pricestyled__Panel-uc7t2j-4 OLjZF" data-test="price-block-panel"><br/>Remboursement annuel des charges par le locataire sur justificatifs.</div></div><div class="Pricestyled__RightColumn-uc7t2j-3 iPHcoG" data-test="price-block-right">Honoraires TTC à la charge du locataire : <strong>689 €</strong><br/><div class="rentHelper__Garantie-sc-1x3dozo-0 AJvvM">Dépôt de garantie : <strong>1 400 €</strong></div><br/><div class="Pricestyled__PriceLinkWrapper-uc7t2j-5 byhXhU"><a class="RealEstatePriceLink__Link-sc-1wcupem-0 wdjdG" href="https://seloger.com/prix-de-l-immo/location/ile-de-france/paris/paris-20eme/pere-lachaise-reunion/48288.htm?bd=DetailPricesToPrices" target="_blank">Consulter tous les prix de l'immobilier à Paris 20ème<div class="RealEst

In [89]:
price_block

In [629]:
description_block = (
    main_tag
    .find_all('div', class_=re.compile('^TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1'))
)

In [630]:
diagnostics_block = main_tag.find_all('div', id='diagnostics')[0]

In [631]:
type(summary_block)

bs4.element.Tag

## Nom du quartier, arrondissement, ville
### Option 1 -  Use summary block

In [632]:
summary_block.find_all('div', class_=re.compile("^Summarystyled__Address-tzuaot-5"))

[<div class="Summarystyled__Address-tzuaot-5 jqlODu">Quartier <!-- -->Gambetta<!-- -->, <!-- -->Paris 20ème</div>]

### Option 2 - Use main block

In [633]:
quartier_block.contents

['L’appartement est situé',
 ' ',
 ' ',
 <strong>à <!-- -->Paris<!-- --> (<!-- -->75020<!-- -->)</strong>,
 ', dans le quartier ',
 ' ',
 'Gambetta',
 ' ',
 '.']

In [634]:
''.join(quartier_block.strong.contents[2:])

'Paris  ( 75020 )'

In [635]:
quartier_block.contents[-3]

'Gambetta'

## Rent price

In [636]:
price_block

<section class="Pricestyled__Container-uc7t2j-1 iUQNoC" data-test="price-block" id="a-propos-de-ce-prix"><div class="Pricestyled__LeftColumn-uc7t2j-2 hJjRAQ"><div class="Pricestyled__Price-uc7t2j-0 cURZmh">Le loyer mensuel est de<div>850 €</div></div><div class="Pricestyled__Panel-uc7t2j-4 OLjZF" data-test="price-block-panel"><strong>Charges comprises</strong>, dont :<br/><div>- Charges forfaitaires :<!-- --> <strong>90 €</strong></div></div></div><div class="Pricestyled__RightColumn-uc7t2j-3 iPHcoG" data-test="price-block-right"><div class="PriceHistorystyled__Container-sc-18jhpbr-0 iSZUrX" data-test="price-history"><div class="PriceHistorystyled__FirstEvolution-sc-18jhpbr-1 jYnlUx"><div class="StyledIcon-vdo5jm-0 bBIEDK Icon__IconElement-sc-1xnn3zd-0 gNSjEe" data-test="sl-ui.icon"><svg viewbox="0 0 512 512"><path d="M484.8 493.8H271.7c-4.6 0-8.3-3.7-8.3-8.3 0-2.2.9-4.3 2.4-5.9l86.6-86.6L21.3 61.9c-11-11-11-28.9 0-39.9s28.9-11 39.9 0l331.1 331.1 86.6-86.6c1.6-1.5 3.7-2.4 5.9-2.4 1.1 0

In [637]:
type(price_block)

bs4.element.Tag

In [638]:
current_price = price_block.find_all('div', 
                                     class_=re.compile('^Pricestyled__Price-uc7t2j-0'))[0].find('div').string

In [639]:
current_price

'850 €'

## Taxes and fees

In [640]:
taxes_and_fees_block = price_block.find_all('div', class_=re.compile('^Pricestyled__Panel-uc7t2j-4'))[0]

In [641]:
assert taxes_and_fees_block.strong.contents[0] == 'Charges comprises'

In [642]:
taxes_and_fees_block

<div class="Pricestyled__Panel-uc7t2j-4 OLjZF" data-test="price-block-panel"><strong>Charges comprises</strong>, dont :<br/><div>- Charges forfaitaires :<!-- --> <strong>90 €</strong></div></div>

In [643]:
taxes_and_fees_block.find_all('div')

[<div>- Charges forfaitaires :<!-- --> <strong>90 €</strong></div>]

In [644]:
taxes_and_fees_block.find_all('div')[0]

<div>- Charges forfaitaires :<!-- --> <strong>90 €</strong></div>

### Honoraires a la charge du locataire

In [645]:
honoraires_block = price_block.find_all('div', class_=re.compile('^Pricestyled__RightColumn-uc7t2j-3'))[0]

In [646]:
honoraires_block

<div class="Pricestyled__RightColumn-uc7t2j-3 iPHcoG" data-test="price-block-right"><div class="PriceHistorystyled__Container-sc-18jhpbr-0 iSZUrX" data-test="price-history"><div class="PriceHistorystyled__FirstEvolution-sc-18jhpbr-1 jYnlUx"><div class="StyledIcon-vdo5jm-0 bBIEDK Icon__IconElement-sc-1xnn3zd-0 gNSjEe" data-test="sl-ui.icon"><svg viewbox="0 0 512 512"><path d="M484.8 493.8H271.7c-4.6 0-8.3-3.7-8.3-8.3 0-2.2.9-4.3 2.4-5.9l86.6-86.6L21.3 61.9c-11-11-11-28.9 0-39.9s28.9-11 39.9 0l331.1 331.1 86.6-86.6c1.6-1.5 3.7-2.4 5.9-2.4 1.1 0 2.2.2 3.2.6 3.1 1.3 5.1 4.3 5.1 7.7v213.1c0 4.6-3.7 8.3-8.3 8.3z"></path></svg></div>-9.6%</div><div class="PriceHistorystyled__Evolutions-sc-18jhpbr-3 gRLKKb"><div class="PriceHistorystyled__FirstEvolutionFull-sc-18jhpbr-2 fCCpRR">Baisse<!-- --> du prix de <span class="PriceHistorystyled__BoldDisplayAmount-sc-18jhpbr-4 kdmxHw global-styles__TextNoWrap-sc-1aeotog-6 dVzJN">90 €</span></div></div></div>Honoraires TTC à la charge du locataire : <stro

In [647]:
honoraires_block.strong.string

'313,65 €'

In [648]:
# try except sur les fluctuations de prix

In [649]:
try:
    price_fluctuations_block = price_block.find_all('div', class_=re.compile('^PriceHistorystyled__Container-sc-18jhpbr-0'))[0]
except IndexError:
    print("Pas d'evolution des prix")

In [650]:
price_change_text= ''.join(price_fluctuations_block.find_all('div', class_=re.compile('^PriceHistorystyled__FirstEvolutionFull-sc-18jhpbr-2'))[0].contents[:3])

In [651]:
price_change_amount = (
    price_fluctuations_block
    .find_all('span',
              class_=re.compile('^PriceHistorystyled__BoldDisplayAmount-sc-18jhpbr-4'))
)[0].contents[0]

In [652]:
price_change_text += price_change_amount

In [653]:
price_change_text

'Baisse  du prix de 90 €'

In [654]:
try:
    deposit_block = price_block.find_all('div', class_=re.compile('^rentHelper__Garantie-sc-1x3dozo-0'))[0]
except IndexError:
    print('No deposit indicated')

In [655]:
deposit_block.strong.contents[0]

'1 520 €'

## Dimensions

In [656]:
dimensions_block = (
    summary_block
    .find_all('div', class_=re.compile("^Summarystyled__TagsWrapper-tzuaot-18"))[0]
    .find_all('div', class_=re.compile("^TagsWithIcon__TagContainer-j1x9om-1"))
)

In [657]:
dimensions_block

[<div class="TagsWithIcon__TagContainer-j1x9om-1 eohXcO"><div class="TagsWithIcon__CurrentIcon-j1x9om-0 bkiamu Icon__IconElement-sc-1xnn3zd-0 jbPwtO" data-test="sl-ui.icon"><svg viewbox="0 0 512 512"><g><path d="M368.6 27.6H132.1c-1.1-.1-2.2-.2-3.2-.2-13.3 0-24.1 10.8-24.1 24.1v335.3c0 9.4 5.4 17.9 13.9 21.9l149.9 69.8c12.1 5.6 26.4.4 32-11.7 1.5-3.2 2.2-6.6 2.2-10.1v-38.4h65.8c19.9 0 36.1-16.2 36.1-36.1V63.7c.1-19.9-16.1-36.1-36.1-36.1zm-89.8 429l-149.9-69.9V51.5l149.9 75v330.1zM380.7 382c0 6.6-5.4 12-12 12h-65.8V126.4c0-9.1-5.1-17.5-13.3-21.5L183.2 51.7h185.5c6.6 0 12 5.4 12 12V382z"></path><path d="M255.4 311.9c6.6 0 12-5.4 12.1-12v-48.2c0-6.7-5.4-12-12-12s-12 5.4-12 12v48.2c-.2 6.6 5.2 12 11.9 12z"></path></g></svg></div><div>1 pièce</div></div>,
 <div class="TagsWithIcon__TagContainer-j1x9om-1 eohXcO"><div class="TagsWithIcon__CurrentIcon-j1x9om-0 bkiamu Icon__IconElement-sc-1xnn3zd-0 jbPwtO" data-test="sl-ui.icon"><svg viewbox="0 0 512 512"><path d="M382 493.6H44.6c-7.2 0-13-5.8-

In [658]:
rooms = dimensions_block[0]

In [659]:
rooms.find_all('div')[-1].string

'1 pièce'

In [660]:
size = dimensions_block[1]

In [661]:
size.find_all('div')[-1].string

'20,91m²'

## L'avis du professionel

In [662]:
avis_du_pro = description_block[0]

In [663]:
avis_du_pro

<div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><div><div class="ShowMoreText__UITextContainer-sc-5ggbbc-0 hCeOyd"><p>- 86, rue de Ménilmontant 75020 Paris - Métro 2 - Ménilmontant Location Meublé - bail étudiant 9 mois Appartement au rez-de-chaussée donnant sur cour, comprenant un espace séjour avec mezzanine, une cuisine ouverte et une salle d'eau avec toilette. Pour toutes visites, merci de contacter Camélia Loyer charges comprises: 850 euros Honoraires: 313.65 euros Dépôt de garantie: 1520 euros COVID-19: Dans le cadre de ce contexte particulier, nos agences Bonjour Oscar restent ouvertes mais l'accès n'est pas permis au public. Nous continuons à travailler et proposons des solutions via rendez-vous téléphonique / conférence en vidéo: - comprendre votre projet d ? Achat ou de location, vous inscrire dans votre fichier, et vous envoyer une visite virtuelle du bien par mail - une aide sur votre financement avec un partenaire - des conseils pour mieux vend

In [664]:
total_desc=''
for s in avis_du_pro.find_all('p'):
    total_desc+=s.string

In [665]:
total_desc

"- 86, rue de Ménilmontant 75020 Paris - Métro 2 - Ménilmontant Location Meublé - bail étudiant 9 mois Appartement au rez-de-chaussée donnant sur cour, comprenant un espace séjour avec mezzanine, une cuisine ouverte et une salle d'eau avec toilette. Pour toutes visites, merci de contacter Camélia Loyer charges comprises: 850 euros Honoraires: 313.65 euros Dépôt de garantie: 1520 euros COVID-19: Dans le cadre de ce contexte particulier, nos agences Bonjour Oscar restent ouvertes mais l'accès n'est pas permis au public. Nous continuons à travailler et proposons des solutions via rendez-vous téléphonique / conférence en vidéo: - comprendre votre projet d ? Achat ou de location, vous inscrire dans votre fichier, et vous envoyer une visite virtuelle du bien par mail - une aide sur votre financement avec un partenaire - des conseils pour mieux vendre ou louer votre bien, avec une première estimation qui sera affinée lors de la visite du bien. -- Récapitulatif documents pour dossier:(locatair

## Amenities

In [672]:
len(description_block)

4

In [678]:
description_block[1].find('div').p

AttributeError: 'NoneType' object has no attribute 'p'

In [684]:
general = description_block[-4].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')

IndexError: list index out of range

In [681]:
general

[<li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Surface de 20,91m²</li>,
 <li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Bâtiment de 5 étages</li>,
 <li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">1 Pièce</li>,
 <li class="GeneralList__Item-sc-9gtpjm-1 gmBAMk">Rez-de-chaussée</li>]

In [599]:
[s.string for s in general]

['Surface de 19,92m²', '1 Pièce', 'Au 6ème étage']

In [600]:
inside = description_block[-3].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')

In [601]:
[s.string for s in inside]

['2 Salles de bain',
 "1 Salle d'eau",
 '1 Toilette',
 'Meublé',
 'Chauffage individuel',
 'Cuisine coin cuisine',
 'Une entrée']

In [602]:
other = description_block[-2].find_all('ul', class_=re.compile('^GeneralList__List-sc-9gtpjm-0'))[0].find_all('li')

In [603]:
[s.string for s in other]

['Calme']

## Diagnostique energetique

In [604]:
diagnostics_block

<div data-test="diagnostics-block" id="diagnostics"><div class="TitledDescription__TitledDescriptionContainer-sc-1r4hqf5-0 fHzMfE"><h3 class="typography__TitleH3-sc-111kv7i-1 jrMlIh">Les diagnostics énergétiques</h3><div class="TitledDescription__TitledDescriptionContent-sc-1r4hqf5-1 koqVoo"><div class="Diagnostics__DiagnosticsContainer-al64ti-2 giLtwp" data-test="diagnostics-content"><div class="Diagnostics__PreviewContainer-al64ti-0 dTGPCg"><p class="Diagnostics__PreviewTitle-al64ti-1 eJGktk" data-test="diagnostics-preview-title">Diagnostic de performance énergétique</p><div class="StatusMention__StatusTitle-sc-1jlgrcu-0 ggNNje">DPE Non communiqué</div></div></div><div class="Diagnostics__DiagnosticsContainer-al64ti-2 giLtwp" data-test="diagnostics-content"><div class="Diagnostics__PreviewContainer-al64ti-0 dTGPCg"><p class="Diagnostics__PreviewTitle-al64ti-1 eJGktk" data-test="diagnostics-preview-title">Indice d'émission de gaz à effet de serre</p><div class="StatusMention__StatusTi

In [605]:
try:
    [energy_diagnostic, ges_diagnostics] = diagnostics_block.find_all('div', class_=re.compile('^Diagnostics__DiagnosticsContainer-al64ti-2'))
except ValueError:
    print('Diagnostique non communique')

In [606]:
energy_diagnostic

<div class="Diagnostics__DiagnosticsContainer-al64ti-2 giLtwp" data-test="diagnostics-content"><div class="Diagnostics__PreviewContainer-al64ti-0 dTGPCg"><p class="Diagnostics__PreviewTitle-al64ti-1 eJGktk" data-test="diagnostics-preview-title">Diagnostic de performance énergétique</p><div class="StatusMention__StatusTitle-sc-1jlgrcu-0 ggNNje">DPE Non communiqué</div></div></div>

In [607]:
type(description_block)

bs4.element.ResultSet

In [608]:
energy_diagnostic.find_all('div', class_=lambda x: x and 'FocusedTile' in x)[0].p.string

IndexError: list index out of range

In [609]:
energy_diagnostic.find_all('div', class_='Preview__PreviewTooltip-sc-1pa12ii-3 hVDlNM')

[]

In [610]:
energy_diagnostic.find('span', class_=re.compile('^Preview__PreviewTooltipValue-sc-1pa12ii-4')).string

AttributeError: 'NoneType' object has no attribute 'string'

In [543]:
energy_diagnostic.find('span', class_=re.compile('^Preview__PreviewTooltipCaption-sc-1pa12ii-5')).string

'kWhEP/m².an'

# Scraping search results
The goal is be to get a list of property URL from one search results URL

In [546]:
sr_url = "https://www.seloger.com/list.htm?ci=750105,750111,940067,940080&idq=133102,133103,133104,133105,133106,133107,133108,133109,133110,133111,133112,133113,133114,133115,133764&idtt=1&idtypebien=1&nb_pieces=1&photo=15&si_meuble=1&surfacemin=20&tri=d_dt_crea"

In [547]:
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,'referer':'https://www.google.com/'}
response = requests.get(sr_url, headers=headers)
soup = BeautifulSoup(response.text,"html.parser")

In [558]:
main_tag = soup.find_all('div', class_='Page__WrapMain-st6q56-1 bgOcDb')[0]

In [571]:
results = main_tag.find_all('div', class_="classified__ContentWrapper-sc-1wmlctl-1 iFOIpl")

In [572]:
len(results)

25

In [581]:
results[0].find_all('a', class_='CoveringLink-a3s3kt-0 dXJclF')[0].get('href')

'https://www.seloger.com/annonces/locations/appartement/paris-11eme-75/leon-blum-folie-regnault/159157123.htm?projects=1&types=1&places=[{ci:750105}|{ci:750111}|{ci:940067}|{ci:940080}|{idq:133102}|{idq:133103}|{idq:133104}|{idq:133105}|{idq:133106}|{idq:133107}|{idq:133108}|{idq:133109}|{idq:133110}|{idq:133111}|{idq:133112}|{idq:133113}|{idq:133114}|{idq:133115}|{idq:133764}]&surface=20/NaN&rooms=1&sort=d_dt_crea&picture=15&qsVersion=1.0&bd=ListToDetail'

In [584]:
for result in results:
    print(result.find_all('a', class_='CoveringLink-a3s3kt-0 dXJclF')[0].get('href'))
    print('\n')

https://www.seloger.com/annonces/locations/appartement/paris-11eme-75/leon-blum-folie-regnault/159157123.htm?projects=1&types=1&places=[{ci:750105}|{ci:750111}|{ci:940067}|{ci:940080}|{idq:133102}|{idq:133103}|{idq:133104}|{idq:133105}|{idq:133106}|{idq:133107}|{idq:133108}|{idq:133109}|{idq:133110}|{idq:133111}|{idq:133112}|{idq:133113}|{idq:133114}|{idq:133115}|{idq:133764}]&surface=20/NaN&rooms=1&sort=d_dt_crea&picture=15&qsVersion=1.0&bd=ListToDetail


https://www.seloger.com/annonces/locations/appartement/paris-11eme-75/nation-alexandre-dumas/155711407.htm?projects=1&types=1&places=[{ci:750105}|{ci:750111}|{ci:940067}|{ci:940080}|{idq:133102}|{idq:133103}|{idq:133104}|{idq:133105}|{idq:133106}|{idq:133107}|{idq:133108}|{idq:133109}|{idq:133110}|{idq:133111}|{idq:133112}|{idq:133113}|{idq:133114}|{idq:133115}|{idq:133764}]&surface=20/NaN&rooms=1&sort=d_dt_crea&picture=15&qsVersion=1.0&bd=ListToDetail


https://www.seloger.com/annonces/locations/appartement/paris-11eme-75/leon-blum-

In [611]:
import random


In [613]:
random.random()*4

1.3844445759704

In [95]:
text = 'blabla'

In [97]:
text+str(0)

'blabla0'

In [98]:
text ='Il semble que vous êtes nombreux à vous connecter depuis ce réseau ...Merci de nous confirmer que vous n’êtes pas un robot : '

In [105]:
text[:66]

'Il semble que vous êtes nombreux à vous connecter depuis ce réseau'